In [3]:
import xml.etree.ElementTree as ET
import pandas as pd
from funciones import get_element_value, combinar_columnas, cambiar_lugar_columnas

# Parsear el archivo XML
# Cambiar el archivo por sys.stdin para NIFI

tree = ET.parse('a.xml')
root = tree.getroot()


filas_sin = []
filas_con = []

# Definir el espacio de nombres
namespace = {'ns': 'http://www.sii.cl/SiiDte'}

Folio = root.find('.//{http://www.sii.cl/SiiDte}Folio').text
RutEmisor = root.find('.//{http://www.sii.cl/SiiDte}RutEmisor').text

detalles = root.findall('.//ns:Detalle', namespace)

for detalle in detalles:
    NroLinDet = get_element_value(detalle, 'NroLinDet')
    NmbItem = get_element_value(detalle, 'NmbItem')
    MontoItem = get_element_value(detalle, 'MontoItem')
    DscItem = get_element_value(detalle, 'DscItem')
    QtyItem = get_element_value(detalle, 'QtyItem')
    
    # Si QtyItem es None, asignar el valor 1
    n_item = QtyItem if QtyItem is not None else '1'

    # Crear una fila para agregar al dataframe
    # Si esta el elemento <DscItem> se agrega al df que contiene las intancias con DscItem
    if DscItem is not None:
        nuevaFila = {
            "rut_emisor": RutEmisor,
            "folio": Folio,
            "n_item": n_item,
            "nmb_item": NmbItem,
            "dsc_item": DscItem,
            "monto_item": MontoItem,
        }
        filas_con.append(nuevaFila)
    # Si no esta el elemento <DscItem> se agrega la fila al df que no tiene 
    else:
        nuevaFila = {
            "rut_emisor": RutEmisor,
            "folio": Folio,
            "n_item": n_item,
            "nmb_item": NmbItem,
            "dsc_item": None,
            "monto_item": MontoItem,
        }
        filas_sin.append(nuevaFila)

# agregar las nuevas filas a los distintos df
df_con = pd.DataFrame(filas_con) if filas_con else pd.DataFrame()  # df con DscItem
df_sin = pd.DataFrame(filas_sin) if filas_sin else pd.DataFrame()  # df sin DscItem

# Procesar df_sin si no está vacío
if not df_sin.empty:
    df_sin = df_sin.drop(columns=['dsc_item'])
    df_sin = df_sin.rename(columns={'nmb_item': 'glosa'})

# Procesar df_con si no está vacío
if not df_con.empty:
    df_con = combinar_columnas(df_con, 'nmb_item', 'dsc_item', 'glosa')
    df_con = cambiar_lugar_columnas(df_con, 'monto_item', 'glosa')

# Juntar los dataframe si ambos no están vacíos
if not df_con.empty and not df_sin.empty:
    df_new = pd.concat([df_con, df_sin], axis=0, ignore_index=False)
elif not df_con.empty:
    df_new = df_con
elif not df_sin.empty:
    df_new = df_sin
else:
    df_new = pd.DataFrame() 


In [5]:
df_new.head()

,rut_emisor,folio,n_item,glosa,monto_item
0,16383874-5,89,1.00,Arriendo Álvaro Casanova,898498
